In [36]:
# created by zayaan. going to test using a basic neural network for the stuff
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
#we want to create a neural network, so we need to import the keras library
from keras.models import Sequential
import tensorflow as tf

In [48]:
csv_file = "sample_data.csv" 
df = pd.read_csv(csv_file, sep=",")
# individual photometry filters
u = df['u']
g = df['g']
r = df['r']
i = df['i']
z = df['z']

# color indices
u_g = u - g
g_r = g - r
r_i = r - i
i_z = i - z

# u_g = u_g[:-19500]
# g_r = g_r[:-19500]
# r_i = r_i[:-19500]
# i_z = i_z[:-19500]

u_g = u_g
g_r = g_r
r_i = r_i
i_z = i_z

# Effective temperature of star
temperature = df['Teff']

color_filters_df = pd.DataFrame()
color_filters_df['u_g'] = u_g
color_filters_df['g_r'] = g_r
color_filters_df['r_i'] = r_i
color_filters_df['i_z'] = i_z

# Effective temperature of star
color_filters_df['temperature'] = df['Teff']

In [49]:
#the input of the model is the color indices, and the output is the temperature.
#lets perform this with a neural network
#we need to split the data into training and testing data
#we will use 80% of the data for training and 20% for testing
#we will also shuffle the data to avoid any bias

#X = color_filters_df[['u_g', 'g_r', 'r_i', 'i_z']].values
X = color_filters_df[['g_r', 'r_i']].values
y = color_filters_df['temperature'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [57]:
#plot the each X vs y
# plt.figure(figsize=(15, 5))
# plt.subplot(1, 4, 1)
# plt.scatter(X_train[:, 0], y_train, s=1, label='train')
# plt.scatter(X_test[:, 0], y_test, s=1, label='test')
# plt.xlabel('u_g')
# plt.ylabel('temperature')
# plt.legend()


In [52]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)), #change input shape based on how many u using
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  #1 neuron output
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [53]:
model.fit(X_train, y_train, epochs=300, batch_size=32, validation_split=0.2)

Epoch 1/300
400/400 [==============================] - 2s 3ms/step - loss: 34354948.0000 - mae: 5563.4165 - val_loss: 27546526.0000 - val_mae: 4758.2227
Epoch 2/300
400/400 [==============================] - 1s 2ms/step - loss: 19061866.0000 - mae: 3695.7261 - val_loss: 12522541.0000 - val_mae: 2972.6543
Epoch 3/300
400/400 [==============================] - 1s 2ms/step - loss: 7398011.5000 - mae: 2221.4895 - val_loss: 3336023.7500 - val_mae: 1419.8141
Epoch 4/300
400/400 [==============================] - 1s 2ms/step - loss: 1883843.2500 - mae: 991.3565 - val_loss: 1315828.2500 - val_mae: 785.6870
Epoch 5/300
400/400 [==============================] - 1s 2ms/step - loss: 1078143.2500 - mae: 696.1162 - val_loss: 1051083.1250 - val_mae: 663.5962
Epoch 6/300
400/400 [==============================] - 1s 2ms/step - loss: 942473.1250 - mae: 623.5115 - val_loss: 987650.6250 - val_mae: 615.5627
Epoch 7/300
400/400 [==============================] - 1s 2ms/step - loss: 906626.0000 - mae: 589.

In [54]:
#print accuracy of model
print(model.evaluate(X_test, y_test))


125/125 [==============================] - 0s 2ms/step - loss: 552645.8125 - mae: 388.9093
[552645.8125, 388.9093017578125]


In [55]:
predictions = model.predict(X_test)

#assign max to a really low number
max = -10000000
#assign min to a really high number
min = 10000000

avg = 0

for i in range(len(X_test)):
    print("Input:", X_test[i])
    print("Expected Output:", y_test[i])
    print("Actual Output:", predictions[i][0])  # predictions[i] is a 2D array, so we access the value with [0]
    print()
    #update max and min accordingly, based on difference in expected and actual output
    if abs(y_test[i] - predictions[i][0]) > max:
        max = abs(y_test[i] - predictions[i][0])
    if abs(y_test[i] - predictions[i][0]) < min:
        min = abs(y_test[i] - predictions[i][0])
    avg += abs(y_test[i] - predictions[i][0])
avg = avg / len(X_test)
print(min, max, avg)

125/125 [==============================] - 0s 2ms/step
Input: [0.49902 0.16507]
Expected Output: 5720
Actual Output: 5539.2603

Input: [0.42466 0.1542 ]
Expected Output: 5651
Actual Output: 5717.9824

Input: [0.59258 0.20528]
Expected Output: 5059
Actual Output: 5295.683

Input: [0.36177 0.12657]
Expected Output: 6000
Actual Output: 5914.396

Input: [1.89165 2.11054]
Expected Output: 3717
Actual Output: 3915.7634

Input: [0.48308 0.19774]
Expected Output: 6000
Actual Output: 5515.0254

Input: [1.6069  1.98742]
Expected Output: 3717
Actual Output: 3776.5488

Input: [0.4614  3.32141]
Expected Output: 3796
Actual Output: 4475.9653

Input: [1.371   0.74839]
Expected Output: 3962
Actual Output: 3880.5688

Input: [0.19687 0.12436]
Expected Output: 7500
Actual Output: 6518.5015

Input: [0.32592 0.15368]
Expected Output: 6500
Actual Output: 5940.2114

Input: [1.65885 1.61367]
Expected Output: 3717
Actual Output: 3689.3428

Input: [0.40481 0.15768]
Expected Output: 5665
Actual Output: 5755.453


In [56]:
r2 = r2_score(y_test, predictions)
print("R2 Score:", r2)

R2 Score: 0.8116170116070175
